In [ ]:

def compute_lombscargle(args):
    time, flux, frequency_chunk = args
    return scipy.signal.lombscargle(time, flux, frequency_chunk, precenter=True, normalize=False)

# Find transit peaks (Lomb-Scargle Periodogram)
def find_transits(time, flux, resolution,period_range):
    '''
    @params
    time: array -> array containing the time values of the kepler dataframe.
    flux: array -> array containing the corrosponding flux values for each time.
    resolution: int -> the number of points to use in the periodogram.

    find the transit peaks, the expected inputs 
    '''
    period = np.linspace(period_range[0], period_range[1], resolution)

    frequency_range = (time[1]-time[0], time[len(time)] -time[0])
    frequency = np.linspace((1/frequency_range[1]), (1/frequency_range[0]), resolution)

    # Split frequency array into chunks for parallel processing
    num_chunks = 12  # Number of processes
    frequency_chunks = np.array_split(frequency, num_chunks)
    period_chunks = np.array_split(period, num_chunks)

    # Use multiprocessing to compute the first Lomb-Scargle periodogram
    with multiprocessing.Pool() as pool:
        power_lomb_1_chunks = pool.map(compute_lombscargle, [(time, flux, chunk) for chunk in frequency_chunks])
    
    # Combine the results from each chunk
    power_lomb_1 = np.concatenate(power_lomb_1_chunks)

    plt.figure(figsize=(6, 6))
    plt.plot(period, power_lomb_1, label="First Lomb-Scargle Periodogram")
    plt.show()

    print('computing second periodogram')
    # Second Lomb-Scargle periodogram on the power spectrum
    with multiprocessing.Pool() as pool:
        power_lomb_2_chunks = pool.map(compute_lombscargle, [(frequency, power_lomb_1, chunk) for chunk in period_chunks])

    power_lomb_2 = np.concatenate(power_lomb_2_chunks)

    return period, power_lomb_2

def run_lomb_scargle_analysis(kepler_dataframe, resolution=5000,period_range=(1, 30)):
    print("Running Lomb-Scargle Periodogram Analysis...")
    period, lomb2 = find_transits(kepler_dataframe["time"], kepler_dataframe["flux"], resolution,period_range)
    
    # Compute the gradient and the second derivative (gradient of the gradient)
    gradient = np.gradient(lomb2, period)
    second_derivative = np.gradient(gradient, period)
    
    # Calculate rolling mean and standard deviation of the gradient
    window_size = 50  # Adjust window size as needed
    rolling_std = pd.Series(gradient).rolling(window=window_size).std().fillna(0)
    
    # Find the point where the gradient stabilizes
    stabilization_index = np.argmax(rolling_std < np.mean(rolling_std))
    gradient_threshold = np.abs(gradient[stabilization_index])
    
    # Determine the second derivative threshold algorithmically
    second_derivative_threshold = np.mean(np.abs(second_derivative)) + 2 * np.std(np.abs(second_derivative))
    
    print(f"Gradient Threshold: {gradient_threshold:.2e}, Second Derivative Threshold: {second_derivative_threshold:.2e}")

    # Plot the gradient
    plt.figure(figsize=(10, 6))
    plt.plot(period, gradient, label="Gradient of Power")
    plt.axhline(0, color='gray', linestyle='--', alpha=0.7)
    plt.axhline(gradient_threshold, color='red', linestyle='--', label='Gradient Threshold')
    plt.xlabel("Period (days)")
    plt.ylabel("Gradient")
    plt.title("Gradient of Lomb-Scargle Power vs Period")
    plt.legend()
    plt.show()

    # Plot the second derivative
    plt.figure(figsize=(10, 6))
    plt.plot(period, second_derivative, label="Second Derivative of Power")
    plt.axhline(0, color='gray', linestyle='--', alpha=0.7)
    plt.xlabel("Period (days)")
    plt.ylabel("Second Derivative")
    plt.title("Second Derivative of Lomb-Scargle Power vs Period")
    plt.legend()
    plt.show()
    
    # Determine regions where both gradient and second derivative are small
    smooth_region_indices = np.where(
        (np.abs(gradient) < gradient_threshold) &
        (np.abs(second_derivative) < second_derivative_threshold)
    )[0]

    # Ensure there are multiple consecutive points in the smooth region
    if len(smooth_region_indices) > 1:
        smooth_start_index = smooth_region_indices[0]  # Start of smooth region
        period_threshold = period[smooth_start_index]
    else:
        period_threshold = np.min(period)  # Fallback if no smooth region is found

    print(f"Excluding peaks before period = {period_threshold:.2f} days")
    
    # Determine peak detection parameters algorithmically
    height = np.mean(lomb2) + 2 * np.std(lomb2)
    distance = resolution // 100
    prominence = np.mean(lomb2) + np.std(lomb2)
    
    # Find initial peaks using scipy's find_peaks with algorithmically determined parameters
    peaks, _ = find_peaks(lomb2, height=height, distance=distance, prominence=prominence)
    peak_pos = period[peaks]
    peak_powers = lomb2[peaks]
    
    # Exclude peaks in the low-period region based on the threshold
    valid_peaks = peak_pos >= period_threshold
    peak_pos = peak_pos[valid_peaks]
    peak_powers = peak_powers[valid_peaks]
    
    print("Lomb-Scargle Periodogram analysis done")

    # Visualize the Lomb-Scargle periodogram and detected peaks
    plt.figure(figsize=(10, 6))
    plt.plot(period, lomb2, label="Lomb-Scargle Periodogram")
    plt.plot(peak_pos, peak_powers, "x", label="Detected Peaks", color="red")
    plt.xlabel("Period (days)")
    plt.ylabel("Power")
    plt.title("Lomb-Scargle Periodogram")
    plt.legend()
    plt.show()

    return peak_pos